<a href="https://colab.research.google.com/github/mounilgarg/dataMining/blob/main/Androsensor2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import csv
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from collections import defaultdict

In [13]:
walking=pd.read_csv("/content/drive/MyDrive/csvData/record3.csv")
del walking['Satellites in range']
walking.head()

,ACCELEROMETER X (m/s²),ACCELEROMETER Y (m/s²),ACCELEROMETER Z (m/s²),PROXIMITY (i),SOUND LEVEL (dB),LOCATION Latitude :,LOCATION Longitude :,LOCATION Altitude ( m),LOCATION Altitude-google ( m),LOCATION Speed ( Kmh),LOCATION Accuracy ( m),LOCATION ORIENTATION (°),Time since start in ms,YYYY-MO-DD HH-MI-SS_SSS
0,-0.41659,3.96001,8.66701,8.0,60.8755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,2022-03-18 12:15:34:930
1,1.97761,-0.35913,9.99819,8.0,64.0218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524,2022-03-18 12:15:35:430
2,0.50278,-0.15802,11.10431,8.0,61.1474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1024,2022-03-18 12:15:35:930
3,0.06704,-0.02394,9.97424,8.0,60.3232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523,2022-03-18 12:15:36:429
4,-0.08619,0.13886,10.23282,8.0,54.9709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,2022-03-18 12:15:36:930


In [15]:
# removing missing values

print(walking.isnull().sum()/len(walking)*100)
# saving missing values in a variable
a = walking.isnull().sum()/len(walking)*100
# saving column names in a variable
variables = walking.columns
variable = [ ]
waste=[ ]
for i in range(0,14):
    if a[i]<=20:   #setting the threshold as 20%
        variable.append(variables[i])
    if a[i]==100:   #noting cols with 100% missing values
        waste.append(variables[i])    
print( )
print(variable)
print(len(variable))
print( )
print(waste)
print(len(waste))

ACCELEROMETER X (m/s²)             0.0
ACCELEROMETER Y (m/s²)             0.0
ACCELEROMETER Z (m/s²)             0.0
PROXIMITY (i)                      0.0
SOUND LEVEL (dB)                   0.0
LOCATION Latitude :              100.0
LOCATION Longitude :             100.0
LOCATION Altitude ( m)           100.0
LOCATION Altitude-google ( m)    100.0
LOCATION Speed ( Kmh)            100.0
LOCATION Accuracy ( m)           100.0
LOCATION ORIENTATION (°)         100.0
Time since start in ms             0.0
YYYY-MO-DD HH-MI-SS_SSS            0.0
dtype: float64

['ACCELEROMETER X (m/s²)', 'ACCELEROMETER Y (m/s²)', 'ACCELEROMETER Z (m/s²)', 'PROXIMITY (i)', 'SOUND LEVEL (dB)', 'Time since start in ms ', 'YYYY-MO-DD HH-MI-SS_SSS']
7

['LOCATION Latitude : ', 'LOCATION Longitude : ', 'LOCATION Altitude ( m)', 'LOCATION Altitude-google ( m)', 'LOCATION Speed ( Kmh)', 'LOCATION Accuracy ( m)', 'LOCATION ORIENTATION (°)']
7


In [16]:
# remove 100 empty valued cols

for col in waste:
  del walking[col]

walking.head()  

,ACCELEROMETER X (m/s²),ACCELEROMETER Y (m/s²),ACCELEROMETER Z (m/s²),PROXIMITY (i),SOUND LEVEL (dB),Time since start in ms,YYYY-MO-DD HH-MI-SS_SSS
0,-0.41659,3.96001,8.66701,8.0,60.8755,24,2022-03-18 12:15:34:930
1,1.97761,-0.35913,9.99819,8.0,64.0218,524,2022-03-18 12:15:35:430
2,0.50278,-0.15802,11.10431,8.0,61.1474,1024,2022-03-18 12:15:35:930
3,0.06704,-0.02394,9.97424,8.0,60.3232,1523,2022-03-18 12:15:36:429
4,-0.08619,0.13886,10.23282,8.0,54.9709,2024,2022-03-18 12:15:36:930


In [17]:
# calculate variance

walking.var()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


ACCELEROMETER X (m/s²)     1.000344e-01
ACCELEROMETER Y (m/s²)     9.019320e-02
ACCELEROMETER Z (m/s²)     1.687383e-01
PROXIMITY (i)              0.000000e+00
SOUND LEVEL (dB)           3.982351e+01
Time since start in ms     2.737626e+09
dtype: float64

In [22]:
# low variance filter
numList=[]
columns=walking.columns
for col in columns:
  if col!='YYYY-MO-DD HH-MI-SS_SSS':
    walking[col]=pd.to_numeric(walking[col])

  if type(col)=='int' or type(col)=='float' or type(col)=='complex' :
    numList.append(col)
numeric = walking[numList]
var = numeric.var()
numeric = numeric.columns
variable = [ ]
for i in range(0,len(var)):
    if var[i]>=-100:   #setting the threshold as -10%
       variable.append(numeric[i])
print(variable)
print(numList)
print(columns)


[]
[]
Index(['ACCELEROMETER X (m/s²)', 'ACCELEROMETER Y (m/s²)',
       'ACCELEROMETER Z (m/s²)', 'PROXIMITY (i)', 'SOUND LEVEL (dB)',
       'Time since start in ms ', 'YYYY-MO-DD HH-MI-SS_SSS'],
      dtype='object')


In [23]:
# high correlation
walking.corr()

,ACCELEROMETER X (m/s²),ACCELEROMETER Y (m/s²),ACCELEROMETER Z (m/s²),PROXIMITY (i),SOUND LEVEL (dB),Time since start in ms
ACCELEROMETER X (m/s²),1.000000,-0.061952,0.167924,NaN,-0.003686,0.073309
ACCELEROMETER Y (m/s²),-0.061952,1.000000,-0.097916,NaN,-0.006702,-0.021405
ACCELEROMETER Z (m/s²),0.167924,-0.097916,1.000000,NaN,-0.031873,0.088829
PROXIMITY (i),NaN,NaN,NaN,NaN,NaN,NaN
SOUND LEVEL (dB),-0.003686,-0.006702,-0.031873,NaN,1.000000,0.087970
Time since start in ms,0.073309,-0.021405,0.088829,NaN,0.087970,1.000000


In [24]:
# remove col with no correlation
del walking['PROXIMITY (i)']
walking.head()

,ACCELEROMETER X (m/s²),ACCELEROMETER Y (m/s²),ACCELEROMETER Z (m/s²),SOUND LEVEL (dB),Time since start in ms,YYYY-MO-DD HH-MI-SS_SSS
0,-0.41659,3.96001,8.66701,60.8755,24,2022-03-18 12:15:34:930
1,1.97761,-0.35913,9.99819,64.0218,524,2022-03-18 12:15:35:430
2,0.50278,-0.15802,11.10431,61.1474,1024,2022-03-18 12:15:35:930
3,0.06704,-0.02394,9.97424,60.3232,1523,2022-03-18 12:15:36:429
4,-0.08619,0.13886,10.23282,54.9709,2024,2022-03-18 12:15:36:930
